# Spark Consumer

TODO: 

-find how to make output to 2 places : save json to hdfs and table to hive

-how come sometimes there are 2 tweets in one output, like:
<pre>
+----+--------------------+  
| key|               value|  
+----+--------------------+  
|null|"@SenWarren Need ...|  
|null|"RT @TrinityResis...|  
+----+--------------------+  
</pre>
-how to consume from a kafka topic everything there is up until now (cursor)  
-how to cope with shit like  <pre>\\ud83c\\uddf5\\ud83c\\uddf8\\u2764\\</pre> instead of arabic chars


In [ ]:
from __future__ import print_function

import sys
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pyspark.sql.functions import split, from_json
from pyspark.sql.types import StructType, StringType, MapType, StructField
from time import time, sleep

In [ ]:
topic = 'TweeterArchive'

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1 pyspark-shell' 

In [ ]:
import pyarrow as pa
fs = pa.hdfs.connect(
    host='localhost', 
    port=8020, 
    user='hdfs', 
    kerb_ticket=None, 
    driver='libhdfs', 
    extra_conf=None)

In [ ]:
hdfs_archive_path = '/tmp/project/archive'    
hdfs_archive_checkpoint_path = hdfs_archive_path + '/sparkcheckpoint'

In [ ]:
#USE WITH CAUTION

#DELETE and recreate the data and checkpoint directories in hdfs 


if fs.exists(hdfs_archive_path):
    fs.delete(hdfs_archive_path,recursive=True)
    print("directory deleted: " + hdfs_archive_path)
else:
    print("no such dir: " + hdfs_archive_path)
#fs.mkdir(hdfs_archive_path)

    
# if fs.exists(hdfs_archive_checkpoint_path):
#     fs.delete(hdfs_archive_checkpoint_path,recursive=True)
    
# else:
#     print("no such dir: " + hdfs_archive_checkpoint_path)
fs.mkdir(hdfs_archive_checkpoint_path)    
print("directory created: " + fs.ls(hdfs_archive_path)[0])
fs.chmod(hdfs_archive_checkpoint_path, 0o777)   
fs.chmod(hdfs_archive_path, 0o777)    
    
    

In [ ]:
spark = SparkSession\
    .builder\
    .appName("StructuredTwitterJsonArchive")\
    .getOrCreate()

In [ ]:

# value schema: { "user": "let's say for now that it is string", "tweet": "let's say for now that it is string" }
schema = StructType().add("user", StringType()).add("tweet", StringType())


In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic) \
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
  .select(col("key").cast("string"), from_json(col("value").cast("string"), schema).alias('value') )

In [ ]:
# Json to HDFS sink with partitioning

targetJsonHDFS = df\
    .writeStream\
    .format("json")\
    .outputMode("append")\
    .option("path", "hdfs://localhost:8020" + hdfs_archive_path)\
    .partitionBy("key")\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_archive_checkpoint_path)\
    .start()
targetJsonHDFS.awaitTermination()

In [ ]:
while True:
    sleep(.1)
    print('\r'+repr(targetJsonHDFS.status)+(' '*100) ,end = '')

In [ ]:
targetJsonHDFS.stop()

In [ ]:
df.stop()

In [ ]:
df.select(from_json())

## Sandbox 
below are cells that I played with to test portions of code


In [ ]:
# schema = StructType([
#                  StructField("user" , StringType(), True),
#                  StructField("tweet", StringType(), True)
#                  ])
# schema

In [ ]:
def foreach_batch_function(df, epoch_id):
    print(df['value'])
    pass
# How to see the output in juppyter:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", topic) \
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
  .writeStream.foreachBatch(foreach_batch_function).start()   

df.awaitTermination()



In [ ]:
df.printSchema()

In [ ]:
 schema = StructType().add("value", StructType().add("user", StringType())
                                            .add("tweet", StringType()))

In [ ]:
schema

In [ ]:
 schema = StructType().add("value", StructType().add("value", StringType()))

In [ ]:
schema

In [ ]:
from pyspark.sql.types import StructType, StringType, MapType, StructField


schema = StructType().add('created_at', StringType(), False).add('id_str', StringType(), False)
schema

In [ ]:
schema = StructType([StructField("value", StringType(), True), True),])

In [ ]:
schema

In [ ]:
schema = StructType([
    StructField("created_at", StringType(), True),
    StructField("id", StringType(), True),
    StructField("text", StringType(), True),
])

In [ ]:
data = spark.read.json(sampleFilePath, schema, multiLine=True)
print(data)

In [ ]:
wat = data.collect()

In [ ]:
wat[0].asDict()

In [ ]:
wat = spark.read.option("multiLine", True).json(sampleFilePath)

In [ ]:
wat.collect()

In [ ]:
jsondf = spark.read.json(Seq(jsonstr).toDS) 

In [ ]:
wat = schema.collect()

In [ ]:
type(wat[0])

In [ ]:
sampleFilePath = "/home/naya/DataEngineerProject/part-00000-7ad71167-a959-4478-929a-6ab7607844d4.c000.json"
sampleFilePath

In [ ]:


with open(sampleFilePath) as f:
    txt = f.read()
    
(txt.__repr__())

json.loads(txt.replace('\\"','\"'))

In [ ]:
sampleFilePath = "hdfs://localhost:8020" +\
                "/tmp/project/archive/key=2019-12-22_10-18/part-00000-7ad71167-a959-4478-929a-6ab7607844d4.c000.json"
sampleFilePath

In [ ]:
from pyspark.sql import SQLContext
sf = sqlContext.read.json(sampleFilePath)

In [ ]:
# json to HDFS sink with trigger

targetJsonHDFS = df\
    .writeStream\
    .format("json")\
    .outputMode("append")\
    .option("path", "hdfs://localhost:8020" + hdfs_archive_path)\
    .trigger(processingTime="5 seconds")\
    .option("checkpointLocation", "hdfs://localhost:8020" + hdfs_archive_checkpoint_path)\
    .start()
targetJsonHDFS.awaitTermination()

In [ ]:
targetJsonHDFS.stop()

In [ ]:
# parquet to HDFS sink example with trigger

targetParquetHDFS = df\
    .writeStream\
    .format("parquet")\
    .outputMode("append")\
    .option("path", "hdfs://localhost:8020/tmp/project")\
    .trigger(processingTime="5 seconds")\
    .option("checkpointLocation", "hdfs://localhost:8020/tmp/sparkcheckpoint/")\
    .start()
targetParquetHDFS.awaitTermination()

In [ ]:
targetParquetHDFS.stop()

In [ ]:
#write parquet files to local linux dir
query = df \
    .writeStream \
    .format("parquet")       \
    .option("path", "/tmp/project")\
    .option("checkpointLocation", "/tmp/sparkcheckpoint/")\
    .outputMode("append")\
    .start()
query.awaitTermination()

In [ ]:
#write parquet files to local linux dir
query = df \
    .writeStream \
    .format("parquet")       \
    .option("path", "/tmp/project")\
    .option("checkpointLocation", "/tmp/sparkcheckpoint/")\
    .outputMode("append")\
    .start()
query.awaitTermination()

In [ ]:
query.stop()

In [ ]:
#output to console
# query = df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()
# query.awaitTermination()

In [ ]:
# words = df.select(
#    explode(
#        split(df.value, " ")
#    ).alias("word")
# )

# Generate running word count
#wordCounts = words.groupBy("word").count()

In [ ]:
# query = wordCounts \
#     .writeStream \
#     .outputMode("complete") \
#     .format("console") \
#     .start()
# query.awaitTermination()